In [63]:
%reset -f
%load_ext autoreload
%autoreload 2

import Odin as backend

[autoreload of Interpolation failed: Traceback (most recent call last):
  File "/home/fynne/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/fynne/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 848, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/fynne/odin/pythonPlayground/Interpolation.py", line 1, in <module>
    from Fields import fieldGovernor
ImportError: cannot import name 'fieldGovernor' from 'Fields' (/home/fynne/odin/pythonPlayground/Fields

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
# validation test for incompressible flow

geom = backend.createGeometry( 'rectangle', [5,2] )
mesh = backend.createMesh( geom, res=1 )

# make just a list, give a name as optional parameter to flow model
myFlowModels = [
    backend.FlowModels.IncompressibleMomentumComp(depVariableName='u', orientation=0, otherVelocityFieldName='v', pressureFieldName='p', kinViscosityName='nu')
]

sim1 = backend.Simulation(myFlowModels, mesh, geom )

myFields = sim1.getFieldRegistry()

print(myFields)

U = myFields['u']
#U.setInternalValue([0,0])
U.u = 0.0
U.v = 0.0

#U.setBoundaryCondition('top', [1,0])
U.setBoundaryCondition('top', 0)
U.setBoundaryCondition('bottom', 0)
U.setBoundaryCondition('left', 1)
U.setBoundaryCondition('right', 0)
myFields['rho'] = 1000.0

sim1.solve('u')

sim1.display(U)

AttributeError: 'cartesian2D' object has no attribute 'shapeStaggered_U'

In [ ]:
# shutting the interpreter down, so I can a fresh instance next time.
# my modules are automatically checked for updates
import os
os._exit(00)